# 第 8 章: PyIceberg 〜軽量な環境における Iceberg 〜

PyIceberg は Apache Iceberg の公式 Python クライアントライブラリであり、分散処理環境や Java の実行環境を必要とせず、Python 環境から Iceberg テーブルを直接操作することを可能にします。本ハンズオンでは、PyIceberg の基本的な使い方を実際に試しながら学んでいきます。

## PyIceberg を使った基本的なテーブル操作を体験する

ここでは、PyIceberg を使ったシンプルなテーブル操作の流れを概観してみましょう。天候の観測データを例に、インストールから設定、テーブル作成、データの追加・読み取り・更新までの基本的な操作を確認します。各要素の詳細な説明は後続のセクションで行います。

In [ ]:
# PyIceberg をインストール
!pip install 'pyiceberg' -q

In [ ]:
# Iceberg カタログへの接続設定
from pyiceberg.catalog import load_catalog

CATALOG_URL     = "http://server:8181"
DEMO_WAREHOUSE  = "s3://amzn-s3-demo-bucket"

props = {
    "type": "rest",
    "uri": CATALOG_URL,
    "warehouse": DEMO_WAREHOUSE,
    "s3.endpoint": "http://minio:9000",
}

catalog = load_catalog(**props)

In [ ]:
# 名前空間の作成
namespace = "pyIceberg"
if not catalog.namespace_exists("pyIceberg"):
    catalog.create_namespace("pyIceberg")

In [ ]:
# テーブルのスキーマ定義
from pyiceberg.schema import Schema
from pyiceberg.types import (
    TimestampType,
    DoubleType,
    StringType,
    NestedField,
    FloatType
)

weather_schema = Schema(
    NestedField(field_id=1, name="observation_time", field_type=TimestampType(), required=False),
    NestedField(field_id=2, name="station_id", field_type=StringType(), required=False),
    NestedField(field_id=3, name="temperature", field_type=DoubleType(), required=False),
    NestedField(field_id=4, name="humidity", field_type=FloatType(), required=False),
    NestedField(field_id=5, name="wind_speed", field_type=FloatType(), required=False),
    NestedField(field_id=6, name="precipitation", field_type=FloatType(), required=False)
)

In [ ]:
table_name = "observations"
table_identifier = f"{namespace}.{table_name}"

# テーブル作成
if catalog.table_exists(table_identifier):
    catalog.purge_table(table_identifier)

table = catalog.create_table(
    identifier=table_identifier,
    schema=weather_schema
)

In [ ]:
import pyarrow as pa
import numpy as np
from datetime import datetime

# サンプルデータの作成
data = pa.Table.from_pylist([
    {
        "observation_time": datetime(2023, 7, 1, 12, 0, 0),
        "station_id": "TOKYO_001",
        "temperature": 28.5,
        "humidity": np.float32(65.0),
        "wind_speed": np.float32(3.2),
        "precipitation": np.float32(0.0)
    },
    {
        "observation_time": datetime(2023, 7, 1, 12, 0, 0),
        "station_id": "OSAKA_002",
        "temperature": 30.2,
        "humidity": np.float32(70.5),
        "wind_speed": np.float32(2.1),
        "precipitation": np.float32(0.0)
    }
])

# テーブルにデータを追加
table.append(data)

In [ ]:
# データ読み取り
scan = table.scan()
scan.to_pandas()

以上が PyIceberg を使った基本的なテーブル操作の流れです。このシンプルな例から、PyIceberg を使うことで Python 環境から直接 Iceberg テーブルを操作するイメージが掴めたと思います。ここからは、より詳細な機能とそれらの活用方法について説明します。

## PyIceberg の基本的な使い方

### PyIceberg のインストールと設定

まずは PyIceberg をインストールします。このハンズオン環境では既にインストールされていますが、通常は以下のコマンドでインストールします。

In [ ]:
!pip install 'pyiceberg' -q

加えて、PyIceberg には特定のデータソースやデータ処理ツールとの統合に使用する追加の依存関係​が用意されています。例えば、テーブルのデータを Pandas で扱い、Iceberg カタログとして AWS Glue Data Catalog を使用する場合には `pandas` と `glue` の依存関係を追加します。インストール時にこれらを指定するには、パッケージ名の後に以下のように括弧 [] で指定します：

| Key             | 説明                                                                                                      |
|-----------------|-----------------------------------------------------------------------------------------------------------|
| hive            | Hive メタストアをサポート                                                                                 |
| hive-kerberos   | Kerberos 環境下での Hive メタストアをサポート                                                               |
| glue            | AWS Glue をサポート                                                                                       |
| dynamodb        | Amazon DynamoDB をサポート                                                                                   |
| sql-postgres    | PostgreSQL をバックエンドとした SQL カタログをサポート                                                      |
| sql-sqlite      | SQLite をバックエンドとした SQL カタログをサポート                                                          |
| pyarrow         | オブジェクトストアとやりとりするための FileIO 実装として PyArrow を利用                                       |
| pandas          | PyArrow と Pandas をインストール                                                                      |
| duckdb          | PyArrow と DuckDB をインストール                                                                      |
| ray             | PyArrow、Pandas、Rayをインストール                                                                        |
| daft            | Daft をインストール                                                                                       |
| polars          | Polars をインストール                                                                                    |
| s3fs            | オブジェクトストアとやりとりするための FileIO 実装として S3FS を利用                                          |
| adlfs           | オブジェクトストアとやりとりするための FileIO 実装として ADLFS を利用                                         |
| snappy          | snappy による Avro 圧縮をサポート                                                                           |
| gcsfs           | オブジェクトストアとやりとりするための FileIO 実装として GCSFS を利用                                         |
| rest-sigv4      | REST カタログのための AWS SIGv4 認証ヘッダ生成をサポート                                                   |

In [ ]:
# Pandasの依存関係をインストールする例
!pip install 'pyiceberg[pandas]' -q

### Iceberg カタログへの接続

PyIceberg で Iceberg テーブルを操作するには、まずテーブルを管理するカタログへの接続を設定します。このハンズオン環境では REST カタログを使用します。

In [ ]:
# Iceberg カタログへの接続設定
from pyiceberg.catalog import load_catalog

CATALOG_URL     = "http://server:8181"
DEMO_WAREHOUSE  = "s3://amzn-s3-demo-bucket"

props = {
    "type": "rest",
    "uri": CATALOG_URL,
    "warehouse": DEMO_WAREHOUSE,
    "s3.endpoint": "http://minio:9000",
}

catalog = load_catalog(**props)

### 名前空間の作成と確認

テーブルを作成する前に、名前空間（namespace）を作成します。名前空間は、テーブルを論理的にグループ化するためのものです。

In [ ]:
# 既存の名前空間を確認
existing_namespaces = catalog.list_namespaces()
print("既存の名前空間:")
for namespace in existing_namespaces:
    print(f"- {'.'.join(namespace)}")

In [ ]:
# 名前空間の作成
namespace = "pyIceberg"
if not catalog.namespace_exists(namespace):
    catalog.create_namespace(namespace)

### テーブルの作成
ここでは、サンプルとなる天候の観測データを格納するテーブルを作成します。

In [ ]:
table_name = "observations"
table_identifier = f"{namespace}.{table_name}"

In [ ]:
# 対象のテーブルが既に存在する場合は削除
if catalog.table_exists(table_identifier):
    catalog.purge_table(table_identifier)

テーブルを作成するために、スキーマを定義します。PyIceberg では、スキーマを定義するために `Schema` クラスを使用します。  
テーブルの列は、 `NestedField` クラスを使用して定義します。`NestedField` クラスは、以下のパラメータを持ちます： 

- field_id: 列の一意な識別子
- name: 列の名前
- field_type: 列のデータ型
- required: 値が必須かどうか

In [ ]:
from pyiceberg.schema import Schema
from pyiceberg.types import (
    TimestampType,
    DoubleType,
    StringType,
    NestedField,
    FloatType
)

# テーブルのスキーマ定義
weather_schema = Schema(
    NestedField(field_id=1, name="observation_time", field_type=TimestampType(), required=False),
    NestedField(field_id=2, name="station_id", field_type=StringType(), required=False),
    NestedField(field_id=3, name="temperature", field_type=DoubleType(), required=False),
    NestedField(field_id=4, name="humidity", field_type=FloatType(), required=False),
    NestedField(field_id=5, name="wind_speed", field_type=FloatType(), required=False),
    NestedField(field_id=6, name="precipitation", field_type=FloatType(), required=False)
)

# スキーマの確認
print(weather_schema)

スキーマとパーティション仕様を定義したら、テーブルを作成します。

In [ ]:
# テーブルの作成
table = catalog.create_table(
    identifier=table_identifier,
    schema=weather_schema
)
print(f"テーブル '{table_identifier}' を作成しました")

### PyArrow スキーマを活用したテーブル作成

テーブルのスキーマを定義する際の別の方法として、 PyIceberg では、 PyArrow のスキーマを直接使用してテーブルを作成することができます。PyArrow スキーマを使用することで、スキーマ定義の手間を省き、既存のデータ構造との一貫性を保ちながらテーブルを作成できます。

In [ ]:
import pyarrow as pa
table_name = "observations"
table_identifier = f"{namespace}.{table_name}"

# PyArrowスキーマの定義
arrow_schema = pa.schema([
    pa.field("observation_time", pa.timestamp('us'), nullable=False),
    pa.field("station_id", pa.string(), nullable=False),
    pa.field("temperature", pa.float64(), nullable=False),
    pa.field("humidity", pa.float32(), nullable=False),
    pa.field("wind_speed", pa.float32(), nullable=False),
    pa.field("precipitation", pa.float32(), nullable=False)
])

# PyArrowスキーマを使用してテーブルを作成
if catalog.table_exists(table_identifier):
    catalog.purge_table(table_identifier)
table = catalog.create_table(
    identifier=table_identifier,
    schema=arrow_schema
)

この使い方は、既存のデータファイルからスキーマを抽出してそのままテーブル作成に利用するシナリオで特に価値を発揮します。例えば、ローカルに保存されたParquetファイルを読み込み、そのスキーマを使用してIcebergテーブルを作成する場合を考えてみましょう。以下のように、PyArrow を使用して Parquet ファイルを読み込み、そのスキーマをそのまま Iceberg テーブルの作成に利用できます。

実験用に適当な Parquet ファイルを作成します。

In [ ]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import os

def generate_simple_weather_data(num_records=30):
    data = {
        "temperature": [round(20 + np.random.normal(0, 5), 1) for _ in range(num_records)],
        "humidity": [round(60 + np.random.normal(0, 10), 1) for _ in range(num_records)],
        "weather_condition": np.random.choice(["SUNNY", "CLOUDY", "RAINY"], size=num_records)
    }
    
    df = pd.DataFrame(data)
    return df

# サンプルデータを生成
output_path = "weather_data.parquet"
weather_df = generate_simple_weather_data(30)

# Parquetファイルとして保存
weather_table = pa.Table.from_pandas(weather_df)
pq.write_table(weather_table, output_path)
print(f"Parquetファイルを保存しました: {output_path}")

In [ ]:
# ファイルのスキーマを確認
parquet_file = pq.ParquetFile(output_path)
print("\nParquetファイルのスキーマ:")
print(parquet_file.schema.to_arrow_schema())

In [ ]:
# 最初の数行を確認
print("\nデータサンプル（最初の5行）:")
first_rows = next(parquet_file.iter_batches(batch_size=5)).to_pandas()
print(first_rows)

この Parquet ファイルを元に、Iceberg テーブルを作ってみましょう。

In [ ]:
import pyiceberg
from pyiceberg.catalog import load_catalog
import pyarrow.parquet as pq

table_name = "observations"
table_identifier = f"{namespace}.{table_name}"

# ローカルのParquetファイルを読み込む
parquet_file = pq.ParquetFile('weather_data.parquet')
# ファイルからスキーマを取得
file_schema = parquet_file.schema.to_arrow_schema()

# 取得したスキーマを使用してIcebergテーブルを作成
if catalog.table_exists(table_identifier):
    catalog.purge_table(table_identifier)
    
table = catalog.create_table(
    identifier=table_identifier,
    schema=file_schema
)

# Parquetファイルからデータを読み込み
table_data = pq.read_table('weather_data.parquet')
# 読み込んだデータをIcebergテーブルに追加
table.append(table_data)

print("Icebergテーブルの作成と読み込みが完了しました")

In [ ]:
table.schema()

In [ ]:
table.scan().to_pandas()

このアプローチの利点は、スキーマを手動で再定義する必要がなく、元のデータ構造がそのまま保持されることです。PyIceberg は PyArrow スキーマからテーブルを作成する際に、自動的に各フィールドに一意のIDを割り当てます。  

これは、Python のデータ分析ライブラリである Pandas や Polars などと組み合わせて使用する際にも便利です。PyArrow スキーマを使用することで、Pandas DataFrame や Polars DataFrame から直接 Iceberg テーブルを作成できるため、データの前処理や変換を行った後に、そのまま Iceberg テーブルとして保存できます。

例えば、Pandas DataFrame から直接 Iceberg テーブルを作成する例を見てみましょう：

In [ ]:
import pandas as pd
import pyarrow as pa
from pyiceberg.utils.config import Config 

table_name = "pandas_observations"
table_identifier = f"{namespace}.{table_name}"

# Pandas DataFrameの作成
df = pd.DataFrame({
    "station_id": ["TOKYO001", "OSAKA002", "NAGOYA003"],
    "temperature": [28.5, 30.2, 27.8],
    "humidity": [65.0, 70.2, 68.5],
    "wind_speed": [3.2, 2.1, 4.5],
    "precipitation": [0.0, 0.0, 2.5]
})

# Pandas DataFrameをPyArrow Tableに変換
arrow_table = pa.Table.from_pandas(df)

# PyArrow Tableのスキーマを使用してIcebergテーブルを作成
if catalog.table_exists(table_identifier):
    catalog.purge_table(table_identifier)
table = catalog.create_table(
    identifier=table_identifier,
    schema=arrow_table.schema
)

# 作成したテーブルにデータを追加
table.append(arrow_table)
# 結果を表示
table.scan().to_pandas()

この例では、Pandas で作成したデータフレームを PyArrow テーブルに変換し、そのスキーマを使って Iceberg テーブルを作成しています。その後、同じデータを Iceberg テーブルに追加しています。このように、Pandas で前処理したデータを PyArrow に変換し、そのスキーマをそのまま使って Iceberg テーブルを作成することで、分析ワークフローをシンプルに保ちながら、Iceberg のデータ管理機能を活用できます。  

## テーブルへの書き込み

PyIceberg では、Apache Arrow を使用してテーブルにデータを書き込めます。主な書き込み操作として、`append`（追加）と `overwrite`（上書き）の2つの方法があります。  

## サンプルデータの準備

In [ ]:
# サンプルテーブル作成
from pyiceberg.schema import Schema
from pyiceberg.types import (
    TimestampType,
    DoubleType,
    StringType,
    NestedField,
    FloatType
)

table_name = "observations"
table_identifier = f"{namespace}.{table_name}"

weather_schema = Schema(
    NestedField(field_id=1, name="observation_time", field_type=TimestampType(), required=False),
    NestedField(field_id=2, name="station_id", field_type=StringType(), required=False),
    NestedField(field_id=3, name="temperature", field_type=DoubleType(), required=False),
    NestedField(field_id=4, name="humidity", field_type=FloatType(), required=False),
    NestedField(field_id=5, name="wind_speed", field_type=FloatType(), required=False),
    NestedField(field_id=6, name="precipitation", field_type=FloatType(), required=False)
)


if catalog.table_exists(table_identifier):
    catalog.purge_table(table_identifier)

table = catalog.create_table(
    identifier=table_identifier,
    schema=weather_schema
)

### データの追加（Append）

`append` は、テーブルにレコードを挿入する際に使用します。

In [ ]:
import pyarrow as pa
import numpy as np
from datetime import datetime

data = pa.Table.from_pylist([
    {
        "observation_time": datetime(2025, 4, 28, 12, 0, 0),
        "station_id": "TOKYO001",
        "temperature": 23.5,
        "humidity": np.float32(65.2),
        "wind_speed": np.float32(4.3),
        "precipitation": np.float32(0.0)
    },
    {
        "observation_time": datetime(2025, 4, 28, 13, 0, 0),
        "station_id": "OSAKA002",
        "temperature": 25.3,
        "humidity": np.float32(58.6),
        "wind_speed": np.float32(3.7),
        "precipitation": np.float32(0.0)
    },
    {
        "observation_time": datetime(2025, 4, 28, 14, 0, 0),
        "station_id": "NAGOYA003",
        "temperature": 22.8,
        "humidity": np.float32(70.5),
        "wind_speed": np.float32(5.2),
        "precipitation": np.float32(1.4)
    }
])

# テーブルに追加
table.append(data)
table.scan().to_pandas()

### データの上書き（Overwrite）

既存データを新しいデータで置き換える場合は、`overwrite` メソッドを使用します。

In [ ]:
updated_data = pa.Table.from_pylist([
    {
        "observation_time": datetime.fromtimestamp(1651234567000000 / 1000000),
        "station_id": "TOKYO001",
        "temperature": 26.8,
        "humidity": np.float32(64.5),
        "wind_speed": np.float32(3.5),
        "precipitation": np.float32(0.2)
    },
    {
        "observation_time": datetime.fromtimestamp(1651234587000000 / 1000000),
        "station_id": "OSAKA002",
        "temperature": 29.1,
        "humidity": np.float32(69.0),
        "wind_speed": np.float32(2.3),
        "precipitation": np.float32(0.0)
    }
])

# テーブルのデータを上書き
table.overwrite(updated_data)
table.scan().to_pandas()

### 条件付き上書き（Partial Overwrite）

特定の条件に一致するデータのみを上書きするには、`overwrite_filter` パラメータを使用します。

In [ ]:
from pyiceberg.expressions import EqualTo

# 条件付き上書き（station_idがTOKYO001のレコードのみを更新）
updated_tokyo = pa.Table.from_pylist([
    {
        "observation_time": datetime.fromtimestamp(1651234567000000 / 1000000),
        "station_id": "TOKYO001",
        "temperature": 27.0,
        "humidity": np.float32(63.0),
        "wind_speed": np.float32(100.8),
        "precipitation": np.float32(0.5)
    }
])

# station_idがTOKYO001のレコードを上書き
table.overwrite(updated_tokyo, overwrite_filter=EqualTo("station_id", "TOKYO001"))
table.scan().to_pandas()

### 動的パーティション上書き（Dynamic Partition Overwrite）

パーティション化されたテーブルでは、データの一部を効率的に更新する必要がしばしば生じます。PyIceberg の `dynamic_partition_overwrite` メソッドは、データに含まれるパーティション値を自動的に検出し、該当するパーティションのみを上書きする機能を提供します。これにより、特定のパーティションのデータのみを更新できます。

この機能は、日次データの更新や、特定のカテゴリのデータを修正する場合など、増分更新のシナリオで特に有用です。
例えば、商品カテゴリごとにパーティション化された販売データテーブルで、電子機器カテゴリの価格データに誤りがあり、それを修正したい場合を考えてみましょう。

In [ ]:
# category　カラムでパーティション化されたテーブルを作成
from pyiceberg.schema import Schema
from pyiceberg.types import LongType, NestedField, StringType, DecimalType
from pyiceberg.partitioning import PartitionSpec, PartitionField
from pyiceberg.transforms import IdentityTransform

table_name = "products"
table_identifier = f"{namespace}.{table_name}"

schema = Schema(
    NestedField(1, "product_id", StringType(), required=False),
    NestedField(2, "category", StringType(), required=False),
    NestedField(3, "product_name", StringType(), required=False),
    NestedField(4, "price", FloatType(), required=False),
    NestedField(5, "stock", LongType(), required=False)
)

# category　カラムによるーパーティションを定義
partition = PartitionSpec(PartitionField(source_id=2, field_id=1001, transform=IdentityTransform(), name="category_identity"))

if catalog.table_exists(table_identifier):
    catalog.purge_table(table_identifier)
table = catalog.create_table(
    table_identifier,
    schema=schema,
    partition_spec=partition
)

In [ ]:
import pyarrow as pa
import numpy as np

# 初期データを追加（電子機器カテゴリの価格に誤りがある）
df = pa.Table.from_pylist([
    {"product_id": "A001", "category": "電子機器", "product_name": "スマートフォン", "price": np.float32(89999.99), "stock": 50},
    {"product_id": "A002", "category": "電子機器", "product_name": "ノートパソコン", "price": np.float32(149999.99), "stock": 30},
    {"product_id": "B001", "category": "家具", "product_name": "ソファ", "price": np.float32(59999.99), "stock": 10},
    {"product_id": "B002", "category": "家具", "product_name": "ダイニングテーブル", "price": np.float32(39999.99), "stock": 15}
])
table.append(df)

ここで、電子機器カテゴリの価格データを修正したいとします。`dynamic_partition_overwrite` メソッドを使用して、電子機器カテゴリのデータのみを更新します。

In [ ]:
# 電子機器カテゴリの正しい価格データを含むテーブル
df_corrected = pa.Table.from_pylist([
    {"product_id": "A001", "category": "電子機器", "product_name": "スマートフォン", "price": np.float32("79999.99"), "stock": 50},
    {"product_id": "A002", "category": "電子機器", "product_name": "ノートパソコン", "price": np.float32("129999.99"), "stock": 30}
])

# 動的パーティション上書きを実行
table.dynamic_partition_overwrite(df_corrected)

この操作により、「電子機器」パーティションのデータのみが新しい価格データで上書きされ、「家具」カテゴリのデータはそのまま保持されます。テーブルの内容を確認すると、以下のようになります。

In [ ]:
table.scan().to_pandas()

このように、`dynamic_partition_overwrite` を使用することで、テーブル全体を再作成することなく、特定のパーティションのデータのみを効率的に更新できます。これは大規模なデータセットを扱う際に特に重要で、更新が必要なデータのみを処理することでリソース使用量を削減し、処理時間を短縮できます。  

また、複数のパーティションを同時に更新することも可能です。更新用のデータに複数のパーティション値が含まれている場合、それらのパーティションすべてが自動的に更新対象となります。  

## アップサート操作（Upsert）

PyIceberg は、既存データの更新と新規データの挿入を1回の操作で行う「アップサート」（upsert）機能をサポートしています。この機能は、テーブルのスキーマで識別子フィールド（identifier field）として指定された列に基づいて、レコードが既に存在する場合は更新し、存在しない場合は新規に挿入します。  
アップサート操作は、増分データの取り込みや、マスターデータの更新などのシナリオで非常に便利です。例えば、定期的に更新されるデータソースからのデータを Iceberg テーブルに取り込む場合、アップサート操作を使用することで、既存のレコードを更新し、新しいレコードを追加することができます。

例えば、都市の人口データを管理するテーブルを考えてみましょう。

In [ ]:
from pyiceberg.schema import Schema
from pyiceberg.types import IntegerType, NestedField, StringType
import pyarrow as pa

table_name = "cities"
table_identifier = f"{namespace}.{table_name}"

# 都市名を識別子フィールドとして指定したスキーマを定義
schema = Schema(
    NestedField(1, "city", StringType(), required=True),
    NestedField(2, "inhabitants", IntegerType(), required=True),
    # 都市名を識別子フィールドして指定
    identifier_field_ids=[1]
)

if catalog.table_exists(table_identifier):
    catalog.purge_table(table_identifier)
table = catalog.create_table(table_identifier, schema=schema)

arrow_schema = pa.schema([
    pa.field("city", pa.string(), nullable=False),
    pa.field("inhabitants", pa.int32(), nullable=False),
])

df = pa.Table.from_pylist([
    {"city": "東京", "inhabitants": 13960000},
    {"city": "大阪", "inhabitants": 2691000},
    {"city": "名古屋", "inhabitants": 2296000},
    {"city": "札幌", "inhabitants": 1952000},
], schema=arrow_schema)

table.append(df)

次に、いくつかの都市の人口データを更新し、新しい都市のデータを追加するアップサート操作を行います。

In [ ]:
# アップサート用のデータを準備
upsert_data = pa.Table.from_pylist([
    # 既存データの更新（名古屋の人口を変更）
    {"city": "名古屋", "inhabitants": 2320000},
    
    # 新規データの挿入（福岡を追加）
    {"city": "福岡", "inhabitants": 1588000},
    
], schema=arrow_schema)

# アップサート操作を実行
result = table.upsert(upsert_data)

# 結果の確認
print(f"更新されたレコード数: {result.rows_updated}")  # 1（名古屋）
print(f"挿入されたレコード数: {result.rows_inserted}")  # 1（福岡）
table.scan().to_pandas()

アップサート操作の結果、名古屋の人口データが更新され、福岡の新しいデータが追加されました。  
アップサート操作の戻り値には、更新されたレコード数（`rows_updated`）と挿入されたレコード数（`rows_inserted`）が含まれており、操作の結果を確認できます。

### レコードの削除（Delete）

`delete` メソッドを使用すると、フィルタ条件に基づいて選択的にレコードを削除できます。  
削除操作は、データクレンジング、プライバシー要件への対応、古いデータの整理など、様々なデータ管理タスクで重要な役割を果たします。PyIceberg では、SQL に似た文字列形式または式オブジェクトを使用して削除条件を指定できます。  

In [ ]:
import pyarrow as pa
import numpy as np
from datetime import datetime
from pyiceberg.schema import Schema
from pyiceberg.types import (
    TimestampType,
    DoubleType,
    StringType,
    NestedField,
    FloatType
)

table_name = "observations"
table_identifier = f"{namespace}.{table_name}"

weather_schema = Schema(
    NestedField(field_id=1, name="observation_time", field_type=TimestampType(), required=False),
    NestedField(field_id=2, name="station_id", field_type=StringType(), required=False),
    NestedField(field_id=3, name="temperature", field_type=DoubleType(), required=False),
    NestedField(field_id=4, name="humidity", field_type=FloatType(), required=False),
    NestedField(field_id=5, name="wind_speed", field_type=FloatType(), required=False),
    NestedField(field_id=6, name="precipitation", field_type=FloatType(), required=False)
)

if catalog.table_exists(table_identifier):
    catalog.purge_table(table_identifier)
table = catalog.create_table(
    identifier=table_identifier,
    schema=weather_schema
)


data = pa.Table.from_pylist([
    {
        "observation_time": datetime(2025, 4, 28, 12, 0, 0),
        "station_id": "TOKYO001",
        "temperature": 19.0,
        "humidity": np.float32(65.2),
        "wind_speed": np.float32(4.3),
        "precipitation": np.float32(0.0)
    },
    {
        "observation_time": datetime(2020, 4, 28, 13, 0, 0),
        "station_id": "OSAKA002",
        "temperature": 25.3,
        "humidity": np.float32(58.6),
        "wind_speed": np.float32(3.7),
        "precipitation": np.float32(0.0)
    },
    {
        "observation_time": datetime(2025, 4, 28, 14, 0, 0),
        "station_id": "NAGOYA003",
        "temperature": 22.8,
        "humidity": np.float32(70.5),
        "wind_speed": np.float32(5.2),
        "precipitation": np.float32(1.4)
    }
])

table.append(data)
table.scan().to_pandas()

In [ ]:
# 文字列形式のフィルタを使用した削除
# 気温が20度未満のレコードを削除
table.delete(delete_filter="temperature < 20.0")

# 複合条件を使用した削除
# 2022年以前の降水量ゼロのレコードを削除
table.delete(delete_filter="observation_time < '2023-01-01T00:00:00' AND precipitation = 0.0")

table.scan().to_pandas()

`pyiceberg.expressions` モジュールには、様々な比較演算子や論理演算子が用意されており、これらを組み合わせて複雑なフィルタ条件を構築できます。  

In [ ]:
from pyiceberg.expressions import LessThan, And, EqualTo, GreaterThanOrEqual
from datetime import datetime

# 特定の期間内の特定の観測所のデータを削除
start_date = datetime(2022, 1, 1)
end_date = datetime(2027, 12, 31)

# pyiceberg.expressionを使用して削除条件を構築
delete_condition = And(
    GreaterThanOrEqual("observation_time", start_date),
    LessThan("observation_time", end_date),
    EqualTo("station_id", "NAGOYA003")
)

table.delete(delete_filter=delete_condition)
table.scan().to_pandas()

削除されたデータは、タイムトラベル機能を使用して過去のスナップショットにアクセスすることで、必要に応じて参照することができます。

In [ ]:
# 削除直前のスナップショットを取得
snapshots = table.inspect.snapshots().to_pandas()
pre_delete_snapshot_id = snapshots.iloc[-2]["snapshot_id"]

# 削除前のデータを参照
table.scan(snapshot_id=pre_delete_snapshot_id).to_pandas()

### データの追加時の注意点

テーブルに追加するデータは、テーブルのスキーマと一致している必要があります。不一致がある場合はエラーが発生します。
特に、PyArrow のスキーマと Iceberg テーブルのスキーマの間で発生しやすい不一致について理解しておくことが重要です。特に注意が必要なのが、タイムスタンプ型と `required` フィールドの扱いです。

Iceberg の `TimestampType` はマイクロ秒精度（'us'）である一方で、PyArrow では秒（'s'）、ミリ秒（'ms'）、マイクロ秒（'us'）、ナノ秒（'ns'）の精度を持つタイムスタンプ型が存在します。PyIceberg では、書き込み時に秒精度とミリ秒精度のタイムスタンプをマイクロ秒精度に変換しますが、ナノ秒精度のタイムスタンプについては設定によってマイクロ秒精度にダウンキャストする必要があります。

In [ ]:
import pyarrow as pa
from datetime import datetime

# タイムスタンプ精度テスト
print("タイムスタンプ精度テスト:")
for precision in ["s", "ms", "us", "ns"]:
    try:
        data = pa.Table.from_pylist([
            {
                "observation_time": datetime(2025, 4, 28, 12, 0, 0),
                "station_id": f"STATION_{precision}",
                "temperature": 23.5, 
                "humidity": np.float32(65.0),
                "wind_speed": np.float32(3.2),
                "precipitation": np.float32(0.0)
            }
        ], schema=pa.schema([
            pa.field("observation_time", pa.timestamp(precision)),
            pa.field("station_id", pa.string()),
            pa.field("temperature", pa.float64()),
            pa.field("humidity", pa.float32()),
            pa.field("wind_speed", pa.float32()),
            pa.field("precipitation", pa.float32())
        ]))
        table.append(data)
        print(f"{precision}: 成功")
    except Exception as e:
        print(f"{precision}: 失敗 - {e}")

PyIceberg のスキーマ定義では、各フィールドに required=True または required=False を指定することで、そのフィールドが必須かどうかを定義します。PyArrow のスキーマでも同様に nullable=False または nullable=True を指定できます。Iceberg テーブルで required=True に設定されたフィールドに対して、PyArrow 側で nullable=True に設定されたカラムを含むテーブルを作成した場合、実際のレコードが null 値を含むかどうかに関わらず、データ追加時にエラーが発生します。  
PyArrow のスキーマでは、デフォルトで nullable=True となるため、Iceberg テーブルのスキーマと一致させるためには、PyArrow のスキーマを定義する際に明示的に nullable=False を指定する必要があります。  


In [ ]:
# サンプルテーブル作成
from pyiceberg.schema import Schema
from pyiceberg.types import (
    TimestampType,
    DoubleType,
    StringType,
    NestedField,
    FloatType
)

table_name = "observations"
table_identifier = f"{namespace}.{table_name}"

weather_schema = Schema(
    NestedField(field_id=1, name="observation_time", field_type=TimestampType(), required=False),
    NestedField(field_id=2, name="station_id", field_type=StringType(), required=True), # required=True を設定
    NestedField(field_id=3, name="temperature", field_type=DoubleType(), required=False),
    NestedField(field_id=4, name="humidity", field_type=FloatType(), required=False),
    NestedField(field_id=5, name="wind_speed", field_type=FloatType(), required=False),
    NestedField(field_id=6, name="precipitation", field_type=FloatType(), required=False)
)


if catalog.table_exists(table_identifier):
    catalog.purge_table(table_identifier)

table = catalog.create_table(
    identifier=table_identifier,
    schema=weather_schema
)

data = pa.Table.from_pylist([
    {
        "observation_time": datetime(2025, 4, 28, 12, 0, 0),
        "station_id": "STATION_001",  # 実際の値はnullではないが、スキーマ定義の不一致でエラーになる
        "temperature": 23.5
    }
], schema=pa.schema([
    pa.field("observation_time", pa.timestamp("us")),  
    pa.field("station_id", pa.string()),  # PyArrowでは、デフォルトで nullable=Trueとなる
    pa.field("temperature", pa.float64())
]))

table.append(data)

In [ ]:
data = pa.Table.from_pylist([
    {
        "observation_time": datetime(2025, 4, 28, 12, 0, 0),
        "station_id": "STATION_001",
        "temperature": 23.5
    }
], schema=pa.schema([
    pa.field("observation_time", pa.timestamp("us")),  
    pa.field("station_id", pa.string(), nullable=False), # 明示的にnullable=Falseを指定する必要がある
    pa.field("temperature", pa.float64())
]))

table.append(data)

## テーブルの参照

`scan` メソッドを使用することで、テーブル全体を対象にデータを取得できます。  

In [ ]:
import pyarrow as pa
import numpy as np
from datetime import datetime
from pyiceberg.schema import Schema
from pyiceberg.types import (
    TimestampType,
    DoubleType,
    StringType,
    NestedField,
    FloatType
)

table_name = "observations"
table_identifier = f"{namespace}.{table_name}"

weather_schema = Schema(
    NestedField(field_id=1, name="observation_time", field_type=TimestampType(), required=False),
    NestedField(field_id=2, name="station_id", field_type=StringType(), required=False),
    NestedField(field_id=3, name="temperature", field_type=DoubleType(), required=False),
    NestedField(field_id=4, name="humidity", field_type=FloatType(), required=False),
    NestedField(field_id=5, name="wind_speed", field_type=FloatType(), required=False),
    NestedField(field_id=6, name="precipitation", field_type=FloatType(), required=False)
)

if catalog.table_exists(table_identifier):
    catalog.purge_table(table_identifier)
table = catalog.create_table(
    identifier=table_identifier,
    schema=weather_schema
)


data = pa.Table.from_pylist([
    {
        "observation_time": datetime(2025, 4, 28, 12, 0, 0),
        "station_id": "TOKYO001",
        "temperature": 19.0,
        "humidity": np.float32(65.2),
        "wind_speed": np.float32(4.3),
        "precipitation": np.float32(0.0)
    },
    {
        "observation_time": datetime(2020, 4, 28, 13, 0, 0),
        "station_id": "OSAKA002",
        "temperature": 25.3,
        "humidity": np.float32(58.6),
        "wind_speed": np.float32(3.7),
        "precipitation": np.float32(0.0)
    },
    {
        "observation_time": datetime(2025, 4, 28, 14, 0, 0),
        "station_id": "NAGOYA003",
        "temperature": 22.8,
        "humidity": np.float32(70.5),
        "wind_speed": np.float32(5.2),
        "precipitation": np.float32(1.4)
    }
])

table.append(data)

In [ ]:
# テーブル全体をスキャン
scan = table.scan()

# スキャン結果をPandasデータフレームとして取得
scan.to_pandas()

PyIceberg では、テーブルからデータを読み取る際に、`scan` メソッドを使用してフィルタリングや列の選択が可能です。フィルタリングは単なる利便性だけでなく、パフォーマンスと効率性の観点からも重要な意味を持ちます。PyIceberg はフィルタ条件をストレージレイヤーまで「プッシュダウン」することで、必要なデータのみを読み込み、メモリ使用量を削減し、処理速度を向上させます。これにより、大規模なテーブルであっても、軽量な環境で効率的に操作できるようになります。  

特に、パーティション列に対するフィルタは「パーティションプルーニング」と呼ばれる最適化が適用され、条件に一致するパーティションのデータファイルのみが読み込まれます。また、データファイルのメタデータに記録された統計情報（列の最小値・最大値など）を活用して、条件に一致しないファイルを読み込み前にスキップする「ファイルスキッピング」も行われます。  

### 文字列形式のフィルタ

文字列形式でフィルタを指定することができます。これは特に動的にフィルタを構築する場合や、簡潔に条件を表現したい場合に便利です

In [ ]:
# 基本的な比較演算子
table.scan(row_filter="temperature >= 25.0").to_pandas()

In [ ]:
# 論理演算子を使った複合条件
string_filter2 = table.scan(
    row_filter="temperature >= 25.0 and humidity > 60.0"
).to_pandas()

In [ ]:
# より複雑な条件
table.scan(
    row_filter="(temperature >= 15.0 and humidity > 60.0) or precipitation > 0"
).to_pandas()

In [ ]:
# IN演算子の使用
table.scan(
    row_filter="station_id in ('TOKYO001', 'OSAKA002', 'NAGOYA003')"
).to_pandas()

In [ ]:
# NULL値の処理
table.scan(
    row_filter="humidity is not null and wind_speed is not null"
).to_pandas()

### 条件式を使用したフィルタリング

`pyiceberg.expressions` モジュールが提供する様々な比較演算子や論理演算子によって、複雑なフィルタ条件を構築できます。これにより、特定の条件に一致するデータのみを効率的に取得できます。  

In [ ]:
from pyiceberg.expressions import (
    GreaterThanOrEqual, LessThan, And, Or, NotNull, IsNull, In, NotIn, EqualTo
)

# 基本的なフィルタ条件
table.scan(
    row_filter=GreaterThanOrEqual("temperature", 25.0),
    selected_fields=("observation_time", "station_id", "temperature")
).to_pandas()

In [ ]:
# 複合条件（25度以上かつ東京の観測所のデータ）
complex_filter = And(
    GreaterThanOrEqual("temperature", 15.0),
    EqualTo("station_id", "TOKYO001")
)
table.scan(row_filter=complex_filter).to_pandas()

In [ ]:
# NULL値の除外（湿度データが欠損していないレコードのみ）
not_null_filter = NotNull("humidity")
table.scan(row_filter=not_null_filter).to_pandas()

In [ ]:
# 値のリストによるフィルタリング（特定の観測所のデータのみ）
stations_filter = In("station_id", ["TOKYO001", "OSAKA002", "NAGOYA003"])
table.scan(row_filter=stations_filter).to_pandas()

### フィルタと列選択の組み合わせ

フィルタと列選択を組み合わせることで、必要な情報のみを効率的に取得できます。

In [ ]:
# 一定以上の気温データの観測時間と気温のみを取得
table.scan(
    row_filter="temperature > 15.0",
    selected_fields=["observation_time", "station_id", "temperature"]
).to_pandas()

In [ ]:
# 特定の条件に基づいて異なる分析を行う
def analyze_weather_patterns(table, min_temp, max_temp):
    # 温度範囲内のデータを取得
    temp_range_data = table.scan(
        row_filter=f"temperature >= {min_temp} and temperature <= {max_temp}",
        selected_fields=["observation_time", "station_id", "temperature", "humidity"]
    ).to_pandas()
    
    # 観測所ごとの集計
    station_stats = temp_range_data.groupby("station_id").agg({
        "temperature": ["mean", "min", "max"],
        "humidity": "mean"
    })
    
    return station_stats

# 温度範囲ごとの分析結果を比較
analyze_weather_patterns(table, 15, 25)

### 結果の制限

スキャン結果の件数を制限する場合は、`limit` パラメータを使用します。これは大規模なテーブルを扱う際に、メモリ使用量を抑えたり、データの探索や検証を効率的に行うために役立ちます。

In [ ]:
# 最大2件に制限したスキャン
limited_scan = table.scan(
    selected_fields=("observation_time", "station_id", "temperature", "humidity", "wind_speed", "precipitation"),
    limit=2
)

limited_scan.to_pandas()

## 発展的な活用法

ここからは、PyIceberg の発展的な機能を活用したデータ管理や分析の方法について説明します。 

### タイムトラベルクエリとタグの実践的活用

Iceberg のタイムトラベル機能は、データの履歴を追跡し、過去の任意の時点のデータを参照できる強力な機能です。PyIceberg では、スナップショット ID やタイムスタンプでタイムトラベルを実現できます。また、タグを使用することで、重要なスナップショットを簡単に参照できるようになります。これにより、データの変更履歴を追跡したり、特定の時点のデータを参照したりすることが容易になります。  

In [ ]:
# サンプルテーブル作成
from pyiceberg.expressions import EqualTo
from pyiceberg.schema import Schema
from pyiceberg.types import (
    TimestampType,
    DoubleType,
    StringType,
    NestedField,
    FloatType
)

table_name = "observations"
table_identifier = f"{namespace}.{table_name}"

weather_schema = Schema(
    NestedField(field_id=1, name="observation_time", field_type=TimestampType(), required=False),
    NestedField(field_id=2, name="station_id", field_type=StringType(), required=False),
    NestedField(field_id=3, name="temperature", field_type=DoubleType(), required=False),
    NestedField(field_id=4, name="humidity", field_type=FloatType(), required=False),
    NestedField(field_id=5, name="wind_speed", field_type=FloatType(), required=False),
    NestedField(field_id=6, name="precipitation", field_type=FloatType(), required=False)
)


if catalog.table_exists(table_identifier):
    catalog.purge_table(table_identifier)

table = catalog.create_table(
    identifier=table_identifier,
    schema=weather_schema
)

# 初期データ書き込み
data = pa.Table.from_pylist([
    {
        "observation_time": datetime(2025, 4, 28, 12, 0, 0),
        "station_id": "TOKYO001",
        "temperature": 19.0,
        "humidity": np.float32(65.2),
        "wind_speed": np.float32(4.3),
        "precipitation": np.float32(0.0)
    },
    {
        "observation_time": datetime(2020, 4, 28, 13, 0, 0),
        "station_id": "OSAKA002",
        "temperature": 25.3,
        "humidity": np.float32(58.6),
        "wind_speed": np.float32(3.7),
        "precipitation": np.float32(0.0)
    },
    {
        "observation_time": datetime(2025, 4, 28, 14, 0, 0),
        "station_id": "NAGOYA003",
        "temperature": 22.8,
        "humidity": np.float32(70.5),
        "wind_speed": np.float32(5.2),
        "precipitation": np.float32(1.4)
    }
])

table.append(data)
# 気温が20度未満のレコードを削除
table.delete(delete_filter="temperature < 20.0")
table.scan().to_pandas()

#### スナップショット ID によるタイムトラベル

まず、テーブルの変更履歴を確認するために、スナップショットの一覧を取得します。  

In [ ]:
# テーブルのスナップショット一覧を取得
table.inspect.snapshots().to_pandas()

特定のスナップショット ID を指定して、その時点のデータを取得できます。以下では、テーブル作成後、初期データが挿入された時点のスナップショット ID を指定して、データを取得しています。

In [ ]:
# スナップショット情報を取得
snapshots = table.inspect.snapshots()
# committed_atでソートするためのインデックスを取得
indices = pa.compute.sort_indices(snapshots["committed_at"])
# 最も古いエントリのインデックスを取得
oldest_index = indices[0].as_py()
# 最も古いsnapshot_idを取得
oldest_snapshot_id = snapshots["snapshot_id"][oldest_index].as_py()

# スナップショットIDでテーブルをスキャン
table.scan(snapshot_id=oldest_snapshot_id).to_pandas()

### タグを使用したスナップショット管理

タグは、特定のスナップショットに任意の名前をつけることで、後から簡単に参照できるようにする仕組みです。これは、ビジネス上の重要なイベントや特定のデータセットを記録する際に便利です。例えば、四半期ごとのレポート作成時のデータスナップショットにタグを付けておくことで、後から簡単にそのデータを参照できます。  

In [ ]:
# サンプルテーブル作成
import pyiceberg
import pyarrow as pa
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pyiceberg.catalog import load_catalog
from pyiceberg.schema import Schema
from pyiceberg.types import (
    TimestampType,
    DoubleType,
    StringType,
    NestedField,
    FloatType
)

table_name = "observations"
table_identifier = f"{namespace}.{table_name}"

weather_schema = Schema(
    NestedField(field_id=1, name="observation_time", field_type=TimestampType(), required=False),
    NestedField(field_id=2, name="station_id", field_type=StringType(), required=False),
    NestedField(field_id=3, name="temperature", field_type=DoubleType(), required=False),
    NestedField(field_id=4, name="humidity", field_type=FloatType(), required=False),
    NestedField(field_id=5, name="wind_speed", field_type=FloatType(), required=False),
    NestedField(field_id=6, name="precipitation", field_type=FloatType(), required=False)
)

if catalog.table_exists(table_identifier):
    catalog.drop_table(table_identifier)

table = catalog.create_table(
    identifier=table_identifier,
    schema=weather_schema
)

# 初期データの書き込み - 2025年4月のデータ
april_data = pa.Table.from_pylist([
    {
        "observation_time": datetime(2025, 4, 1, 12, 0, 0),
        "station_id": "TOKYO001",
        "temperature": 19.0,
        "humidity": np.float32(65.2),
        "wind_speed": np.float32(4.3),
        "precipitation": np.float32(0.0)
    },
    {
        "observation_time": datetime(2025, 4, 1, 13, 0, 0),
        "station_id": "OSAKA002",
        "temperature": 25.3,
        "humidity": np.float32(58.6),
        "wind_speed": np.float32(3.7),
        "precipitation": np.float32(0.0)
    },
    {
        "observation_time": datetime(2025, 4, 1, 14, 0, 0),
        "station_id": "NAGOYA003",
        "temperature": 22.8,
        "humidity": np.float32(70.5),
        "wind_speed": np.float32(5.2),
        "precipitation": np.float32(1.4)
    }
])
table.append(april_data)

In [ ]:
# 初期スナップショットに「april_data」というタグをつける
# スナップショットIDを取得
snapshots = table.snapshots()
initial_snapshot_id = table.current_snapshot().snapshot_id

# タグを作成
with table.manage_snapshots() as snapshots:
    snapshots.create_tag(initial_snapshot_id, "april_data")
    print(f"初期スナップショット {initial_snapshot_id} に 'april_data' タグを付けました")

In [ ]:
# 5月のデータを追加
may_data = pa.Table.from_pylist([
    {
        "observation_time": datetime(2025, 5, 1, 12, 0, 0),
        "station_id": "TOKYO001",
        "temperature": 23.5,
        "humidity": np.float32(60.0),
        "wind_speed": np.float32(3.8),
        "precipitation": np.float32(0.0)
    },
    {
        "observation_time": datetime(2025, 5, 1, 13, 0, 0),
        "station_id": "OSAKA002",
        "temperature": 29.1,
        "humidity": np.float32(55.3),
        "wind_speed": np.float32(4.2),
        "precipitation": np.float32(0.0)
    }
])
table.append(may_data)

# 5月のデータを追加した後のスナップショットに「may_data」タグをつける
may_snapshot_id = table.current_snapshot().snapshot_id
with table.manage_snapshots() as snapshots:
    snapshots.create_tag(may_snapshot_id, "may_data")
    print(f"5月データ追加後のスナップショット {may_snapshot_id} に 'may_data' タグを付けました")

In [ ]:
# 現在のテーブルの状態を確認
print("\n現在のテーブルデータ（4月と5月のデータ両方含む）:")
table.scan().to_pandas()

In [ ]:
# タグ付けされたスナップショットの一覧を表示
print("\nテーブルのタグとスナップショット:")
refs = table.metadata.refs
for ref_name, ref_details in refs.items():
    print(f"タグ: {ref_name}, スナップショットID: {ref_details.snapshot_id}")

In [ ]:
# タグを使用して特定のスナップショットのデータを取得
# タグが参照するスナップショットIDを取得
april_snapshot_id = None
may_snapshot_id = None

if "april_data" in refs:
    april_snapshot_id = refs["april_data"].snapshot_id
if "may_data" in refs:
    may_snapshot_id = refs["may_data"].snapshot_id

# 「april_data」タグを使用して4月のデータだけを取得
print("\n'april_data'タグを使用して4月のデータを取得:")
table.scan(snapshot_id=april_snapshot_id).to_pandas()

In [ ]:
# 「may_data」タグを使用して5月のデータを含む状態を取得
print("\n'may_data'タグを使用して5月までのデータを取得:")
table.scan(snapshot_id=may_snapshot_id).to_pandas()

### 発展的なテーブル定義

PyIcebergでは、テーブルの作成時にパーティショニングやソート順序などの高度な設定を行うことができます。これらの機能を使うことで、テーブルの物理的な構造を制御し、クエリパフォーマンスの最適化に役立てられます。  

#### パーティションの定義

パーティショニングは、データを論理的なグループに分割する機能です。PyIcebergでは、様々なパーティション変換を使用して、元のデータ列からパーティション値を生成します。  

パーティション仕様を定義するには、`PartitionSpec`クラスと`PartitionField`クラスを使用します：  

In [ ]:
import pyiceberg
import pyarrow as pa
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pyiceberg.partitioning import PartitionSpec, PartitionField
from pyiceberg.transforms import (
    IdentityTransform,
    DayTransform,
)
from pyiceberg.catalog import load_catalog
from pyiceberg.schema import Schema
from pyiceberg.types import (
    TimestampType,
    DoubleType,
    StringType,
    NestedField,
    FloatType
)

table_name = "observations"
table_identifier = f"{namespace}.{table_name}"

schema = Schema(
    NestedField(field_id=1, name="observation_time", field_type=TimestampType(), required=False),
    NestedField(field_id=2, name="station_id", field_type=StringType(), required=False),
    NestedField(field_id=3, name="temperature", field_type=DoubleType(), required=False),
    NestedField(field_id=4, name="humidity", field_type=FloatType(), required=False),
    NestedField(field_id=5, name="wind_speed", field_type=FloatType(), required=False),
    NestedField(field_id=6, name="precipitation", field_type=FloatType(), required=False)
)

# パーティション仕様の定義
partition_spec = PartitionSpec(
    # observation_timeフィールドを日単位でパーティション化
    PartitionField(source_id=1, field_id=1000, transform=DayTransform(), name="observation_day"),
    # station_idフィールドをそのままパーティションとして使用
    PartitionField(source_id=2, field_id=1001, transform=IdentityTransform(), name="station_id")
)

if catalog.table_exists(table_identifier):
    catalog.drop_table(table_identifier)

# パーティション仕様を指定してテーブル作成
table = catalog.create_table(
    identifier=table_identifier,
    schema=schema,
    partition_spec=partition_spec
)
print("パーティション定義：")
table.spec()

`PartitionField` は、パーティション化するフィールドを定義するためのクラスで、以下のパラメータを指定します：
- `source_id`: 元のフィールドのID
- `field_id`: パーティションフィールドのID
- `transform`: パーティション化に使用する変換（Transform）
- `name`: パーティションフィールドの名前

`PartitionSpec` は、複数の `PartitionField` を組み合わせてパーティション仕様を定義するためのクラスです。これにより、複数のフィールドを組み合わせた複雑なパーティション化が可能になります。  

PyIcebergでは、以下の Transform がサポートされています：  

- `IdentityTransform`: 元の値をそのまま使用します
- `BucketTransform`: 値のハッシュに基づいてデータを指定した数のバケットに分割します
- `TruncateTransform`: 文字列の先頭から指定した長さの部分を使用します
- `YearTransform`: タイムスタンプから年を抽出します
- `MonthTransform`: タイムスタンプから月を抽出します
- `DayTransform`: タイムスタンプから日を抽出します
- `HourTransform`: タイムスタンプから時間を抽出します

### ソート順序の定義

ソート順序は、データファイル内のレコードの配置を制御します。ソート順序を定義するには、`SortOrder`クラスと`SortField`クラスを使用します。  

In [ ]:
from pyiceberg.table.sorting import SortOrder, SortField
from pyiceberg.transforms import IdentityTransform
import pyiceberg
import pyarrow as pa
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pyiceberg.partitioning import PartitionSpec, PartitionField
from pyiceberg.transforms import (
    IdentityTransform,
    DayTransform,
)
from pyiceberg.catalog import load_catalog
from pyiceberg.schema import Schema
from pyiceberg.types import (
    TimestampType,
    DoubleType,
    StringType,
    NestedField,
    FloatType
)

table_name = "observations"
table_identifier = f"{namespace}.{table_name}"

schema = Schema(
    NestedField(field_id=1, name="observation_time", field_type=TimestampType(), required=False),
    NestedField(field_id=2, name="station_id", field_type=StringType(), required=False),
    NestedField(field_id=3, name="temperature", field_type=DoubleType(), required=False),
    NestedField(field_id=4, name="humidity", field_type=FloatType(), required=False),
    NestedField(field_id=5, name="wind_speed", field_type=FloatType(), required=False),
    NestedField(field_id=6, name="precipitation", field_type=FloatType(), required=False)
)

# 単一フィールドによるソート順序の定義
simple_sort_order = SortOrder(
    SortField(source_id=2, transform=IdentityTransform())  # station_idでソート
)

if catalog.table_exists(table_identifier):
    catalog.drop_table(table_identifier)

# ソート順序を指定してテーブルを作成
table = catalog.create_table(
    identifier=table_identifier,
    schema=schema,
    sort_order=simple_sort_order
)
print("ソート順定義：")
table.metadata.sort_orders

In [ ]:
from pyiceberg.table.sorting import SortOrder, SortField
from pyiceberg.transforms import IdentityTransform
import pyiceberg
import pyarrow as pa
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pyiceberg.partitioning import PartitionSpec, PartitionField
from pyiceberg.catalog import load_catalog
from pyiceberg.schema import Schema
from pyiceberg.types import (
    TimestampType,
    DoubleType,
    StringType,
    NestedField,
    FloatType
)

table_name = "observations"
table_identifier = f"{namespace}.{table_name}"

schema = Schema(
    NestedField(field_id=1, name="observation_time", field_type=TimestampType(), required=False),
    NestedField(field_id=2, name="station_id", field_type=StringType(), required=False),
    NestedField(field_id=3, name="temperature", field_type=DoubleType(), required=False),
    NestedField(field_id=4, name="humidity", field_type=FloatType(), required=False),
    NestedField(field_id=5, name="wind_speed", field_type=FloatType(), required=False),
    NestedField(field_id=6, name="precipitation", field_type=FloatType(), required=False)
)

# 複数フィールドによるソート順序の定義
multi_field_sort_order = SortOrder(
    SortField(source_id=2, transform=IdentityTransform()),  # まずstation_idでソート
    SortField(source_id=1, transform=IdentityTransform())   # 次にobservation_timeでソート
)

if catalog.table_exists(table_identifier):
    catalog.drop_table(table_identifier)

# ソート順序を指定してテーブルを作成
table = catalog.create_table(
    identifier=table_identifier,
    schema=schema,
    sort_order=multi_field_sort_order
)
print("ソート順定義：")
table.metadata.sort_orders

`SortField`のパラメータは以下の通りです：  

- `source_id`: ソートに使用するフィールドのID
- `transform`: フィールド値に適用する変換（パーティションと同様の変換が使用可能）
- `direction`: ソート方向（デフォルトは昇順）
- `null_order`: NULL値の順序（デフォルトはNULL値を最後に配置）

ソート方向を明示的に指定するには、`SortDirection` を使用します：  

NULL値の順序も制御できます。

In [ ]:
from pyiceberg.table.sorting import SortOrder, SortField
from pyiceberg.transforms import IdentityTransform
import pyiceberg
import pyarrow as pa
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pyiceberg.partitioning import PartitionSpec, PartitionField
from pyiceberg.catalog import load_catalog
from pyiceberg.schema import Schema
from pyiceberg.table.sorting import NullOrder
from pyiceberg.types import (
    TimestampType,
    DoubleType,
    StringType,
    NestedField,
    FloatType
)

table_name = "observations"
table_identifier = f"{namespace}.{table_name}"

schema = Schema(
    NestedField(field_id=1, name="observation_time", field_type=TimestampType(), required=False),
    NestedField(field_id=2, name="station_id", field_type=StringType(), required=False),
    NestedField(field_id=3, name="temperature", field_type=DoubleType(), required=False),
    NestedField(field_id=4, name="humidity", field_type=FloatType(), required=False),
    NestedField(field_id=5, name="wind_speed", field_type=FloatType(), required=False),
    NestedField(field_id=6, name="precipitation", field_type=FloatType(), required=False)
)

# NULL値の順序を指定したソート
null_aware_sort = SortOrder(
    SortField(
        source_id=4,                           # humidity（欠測値があり得る）
        transform=IdentityTransform(),
        null_order=NullOrder.NULLS_FIRST      # NULL値を先頭に配置
    )
)

if catalog.table_exists(table_identifier):
    catalog.drop_table(table_identifier)

# ソート順序を指定してテーブルを作成
table = catalog.create_table(
    identifier=table_identifier,
    schema=schema,
    sort_order=null_aware_sort
)
print("ソート順定義：")
table.sort_order()

パーティションと同様に、ソートフィールドにも変換を適用できます： 

In [ ]:
from pyiceberg.table.sorting import SortOrder, SortField
from pyiceberg.transforms import IdentityTransform
import pyiceberg
import pyarrow as pa
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pyiceberg.partitioning import PartitionSpec, PartitionField
from pyiceberg.catalog import load_catalog
from pyiceberg.schema import Schema
from pyiceberg.table.sorting import SortDirection
from pyiceberg.types import (
    TimestampType,
    DoubleType,
    StringType,
    NestedField,
    FloatType
)

table_name = "observations"
table_identifier = f"{namespace}.{table_name}"

schema = Schema(
    NestedField(field_id=1, name="observation_time", field_type=TimestampType(), required=False),
    NestedField(field_id=2, name="station_id", field_type=StringType(), required=False),
    NestedField(field_id=3, name="temperature", field_type=DoubleType(), required=False),
    NestedField(field_id=4, name="humidity", field_type=FloatType(), required=False),
    NestedField(field_id=5, name="wind_speed", field_type=FloatType(), required=False),
    NestedField(field_id=6, name="precipitation", field_type=FloatType(), required=False)
)

# 変換を適用したソートフィールド
transformed_sort = SortOrder(
    SortField(
        source_id=1,                           # observation_time
        transform=DayTransform(),              # 日付部分のみでソート
        direction=SortDirection.DESC           # 新しい日付から古い日付の順
    )
)

if catalog.table_exists(table_identifier):
    catalog.drop_table(table_identifier)

# ソート順序を指定してテーブルを作成
table = catalog.create_table(
    identifier=table_identifier,
    schema=schema,
    sort_order=transformed_sort
)
print("ソート順定義：")
table.sort_order()

## Python のデータ分析ライブラリとの連携

### Pandas との連携

Pandas は Python で広く使用されているデータ分析ライブラリです。DataFrame という表形式のデータ構造を提供し、データの操作、変換、可視化、統計分析などの機能を備えています。直感的な API と豊富な機能により、データサイエンティストやアナリストの標準ツールとなっています。

PyIceberg と Pandas を組み合わせることで、Iceberg の高度なデータ管理機能と Pandas の使いやすいデータ分析機能を両立できます。PyIceberg が行うフィルタリングはストレージレベルで適用されるため、必要なデータのみを効率的に読み込み、メモリ使用量を最適化できます。

In [ ]:
# サンプルテーブル作成
from pyiceberg.expressions import EqualTo
from pyiceberg.schema import Schema
from pyiceberg.types import (
    TimestampType,
    DoubleType,
    StringType,
    NestedField,
    FloatType
)

table_name = "observations"
table_identifier = f"{namespace}.{table_name}"

weather_schema = Schema(
    NestedField(field_id=1, name="observation_time", field_type=TimestampType(), required=False),
    NestedField(field_id=2, name="station_id", field_type=StringType(), required=False),
    NestedField(field_id=3, name="temperature", field_type=DoubleType(), required=False),
    NestedField(field_id=4, name="humidity", field_type=FloatType(), required=False),
    NestedField(field_id=5, name="wind_speed", field_type=FloatType(), required=False),
    NestedField(field_id=6, name="precipitation", field_type=FloatType(), required=False)
)


if catalog.table_exists(table_identifier):
    catalog.purge_table(table_identifier)

table = catalog.create_table(
    identifier=table_identifier,
    schema=weather_schema
)

data = pa.Table.from_pylist([
    {
        "observation_time": datetime(2025, 4, 28, 12, 0, 0),
        "station_id": "TOKYO001",
        "temperature": 19.0,
        "humidity": np.float32(65.2),
        "wind_speed": np.float32(4.3),
        "precipitation": np.float32(0.0)
    },
    {
        "observation_time": datetime(2020, 4, 28, 13, 0, 0),
        "station_id": "OSAKA002",
        "temperature": 25.3,
        "humidity": np.float32(58.6),
        "wind_speed": np.float32(3.7),
        "precipitation": np.float32(0.0)
    },
    {
        "observation_time": datetime(2025, 4, 28, 14, 0, 0),
        "station_id": "NAGOYA003",
        "temperature": 22.8,
        "humidity": np.float32(70.5),
        "wind_speed": np.float32(5.2),
        "precipitation": np.float32(1.4)
    }
])

table.append(data)

In [ ]:
import pandas as pd

# テーブルをスキャンしてPandasデータフレームに変換
df = table.scan(
    row_filter="temperature >= 25.0",
    selected_fields=("observation_time", "station_id", "temperature", "humidity", "precipitation")
).to_pandas()

# Pandasの機能を使用したデータ分析
avg_temp_by_station = df.groupby("station_id")["temperature"].mean()
print("観測所ごとの平均気温:")
print(avg_temp_by_station)

In [ ]:
# 日付ごとの平均気温と湿度
df["date"] = df["observation_time"].dt.date
daily_stats = df.groupby("date").agg({
    "temperature": ["mean", "min", "max"],
    "humidity": ["mean", "min", "max"]
})

print("日付ごとの気象統計:")
print(daily_stats)

### Polars と PyIceberg の連携

Polars は、高性能な列指向データフレームライブラリです。Rust で実装されており、遅延評価（Lazy Evaluation）による最適化などを用いた、高速なパフォーマンスが特徴です。  
遅延評価とは、実際にデータが必要になるまで計算を実行せず、最適な実行計画を自動的に構築する機能です。これにより、PyIceberg からの大規模データでも効率的な処理が可能になります。フィルタ条件の最適化や不要な列の読み込み回避などが自動的に行われ、メモリ使用量とパフォーマンスが改善されます。  

In [ ]:
!pip install pyiceberg[polars] -q

In [ ]:
import polars as pl
# テーブルをスキャンしてPolarsデータフレームに変換
polars_df = table.scan(
    row_filter="observation_time >= 1651234500000000",
    selected_fields=("observation_time", "station_id", "temperature", "humidity", "precipitation")
).to_polars()

# Polarsの機能を使用したデータ分析
polars_df.group_by("station_id").agg([
    pl.col("temperature").mean().alias("avg_temperature"),
    pl.col("precipitation").sum().alias("total_precipitation")
])

また、LazyFrame を使用した遅延評価も可能です：

In [ ]:
# テーブルからPolars LazyFrameを取得
lazy_frame = table.to_polars()

# 遅延評価によるフィルタリングと演算
result = (
    lazy_frame
    .filter(pl.col("temperature") > 25.0)
    .select(["station_id", "temperature", "humidity"])
    .group_by("station_id")
    .agg([
        pl.col("temperature").mean().alias("avg_temperature"),
        pl.col("humidity").mean().alias("avg_humidity")
    ])
    .collect()  # ここで実際の計算が実行される
)
result

### DuckDB と PyIceberg の連携

DuckDB は、分析ワークロード向けに設計された高性能な組み込み型 SQL データベースエンジンです。メモリ内で高速に動作し、大規模なデータセットも効率的に処理できます。Python 環境との統合が容易で、SQL の表現力を活かした分析が可能です。

PyIceberg と DuckDB の連携により、Iceberg テーブルに対して SQL クエリを実行できます。`to_duckdb()` メソッドを使用して、テーブルを DuckDB に変換するだけで SQL 分析が可能になります。

In [ ]:
!pip install pyiceberg[duckdb] -q

In [ ]:
# テーブルをDuckDBに変換
con = table.scan().to_duckdb(table_name="weather_observations")

# SQLクエリの実行
result = con.execute("""
    SELECT 
        station_id, 
        AVG(temperature) as avg_temperature, 
        SUM(precipitation) as total_precipitation 
    FROM weather_observations 
    WHERE temperature > 25.0 
    GROUP BY station_id
    ORDER BY total_precipitation DESC
""").fetchdf()  # pandas DataFrameとして結果を取得
result

PyIceberg のフィルタ条件と DuckDB の SQL を組み合わせることで、効率的なデータ処理が可能です。例えば、パーティション列に対するフィルタを PyIceberg で適用し、詳細な分析を DuckDB で行うことができます。

In [ ]:
# PyIcebergでフィルタリングしてからDuckDBに変換
filtered_scan = table.scan(
    row_filter="observation_time >= '2023-01-01T00:00:00' and observation_time < '2026-02-01T00:00:00'",
    selected_fields=["observation_time", "station_id", "temperature", "humidity"]
)

# DuckDBで集計分析
con = filtered_scan.to_duckdb(table_name="january_observations")
monthly_stats = con.execute("""
    SELECT 
        station_id,
        AVG(temperature) as avg_temp,
        MIN(temperature) as min_temp,
        MAX(temperature) as max_temp,
        AVG(humidity) as avg_humidity
    FROM january_observations
    GROUP BY station_id
""").fetchdf()
monthly_stats

## PyIceberg の設定を管理する

PyIceberg では、`.pyiceberg.yaml` という設定ファイルを使用することで、カタログへの接続設定を管理できます。この設定ファイルは PyIceberg のさまざまな動作を制御するために使用され、ユーザーが定義した設定はライブラリの初期化時に読み込まれます。これにより、PyIceberg を使用する際の設定を一元管理でき、複数のプロジェクトや環境での再利用が容易になります。  
`.pyiceberg.yaml` ファイルは YAML 形式で記述され、カタログの設定やデフォルトのテーブルプロパティなどを指定できます。設定ファイルは `PYICEBERG_HOME` 環境変数で指定されたディレクトリ、ホームディレクトリ、またはワーキングディレクトリのいずれかに配置できます。  

本ハンズオン環境には、以下の `.pyiceberg.yaml` が配置されています。

In [ ]:
!cat /home/jovyan/.pyiceberg.yaml

この設定ファイルでは、`catalog` セクションでカタログの設定を定義しています。`default` はカタログの名前で、`type` はカタログの種類を指定します。ここでは REST カタログを使用しており、`uri` にはハンズオン環境の REST カタログサーバーの URL を指定しています。また、ストレージ(minio)に関する設定も含まれています。

この設定ファイルによって、ハンズオン環境では、カタログへの接続を定義する際に、`default` を指定するだけで接続できます。

In [ ]:
# Iceberg カタログへの接続設定
from pyiceberg.catalog import load_catalog
catalog = load_catalog("default")

Icebergテーブルの動作に関するプロパティも `.pyiceberg.yaml` で設定できます。例えば、以下のような設定が可能です。  

- write.format.default: データファイルのデフォルトフォーマット。parquet, avro, orc から選択。デフォルトは parquet
- write.parquet.compression-codec: Parquetの圧縮アルゴリズム。uncompressed, zstd, gzip, snappyから選択。デフォルトは zstd
- write.object-storage.enabled: ストレージへの書き込み時に、プレフィックスの分散による最適化をするかどうか。デフォルトは False

これらの設定は、PyIceberg でのテーブル操作時のデフォルト値として使用されます

設定可能なプロパティの詳細は、PyIceberg の公式ドキュメントを参照してください。  
https://py.iceberg.apache.org/configuration/

設定ファイルを使用する代わりに、環境変数を通じて設定を提供することも可能です。これらの環境変数は`PYICEBERG_`プレフィックスから始まり、続いてYAML構造に対応する名前を使用します。階層構造は二重アンダースコア`__`で表現され、ダッシュ`-`は単一アンダースコア`_`として表されます。例えば、`PYICEBERG_CATALOG__DEFAULT__S3__ACCESS_KEY_ID` は、YAML ファイルの `catalog.default.s3.access-key-id` に相当します。

## Python CLI

PyIceberg には、Iceberg テーブルのメタデータを参照・管理するためのコマンドラインインターフェース（CLI）が含まれています。この CLI は主にテーブルの一覧表示、スキーマの確認、テーブルプロパティの管理などのメタデータ操作に特化しており、テーブルデータの読み書きは行いません。Python API を使用せずとも、コマンドラインから直接 Iceberg テーブルのメタデータを操作できるため、テーブル構造の調査や設定の確認・変更に役立ちます。また、運用系のスクリプトやデータパイプラインの一部に組み込んで活用できます。

### 基本的な使い方
PyIceberg CLI の基本的な構文は以下の通りです：  

```bash
pyiceberg [オプション] コマンド [引数]
```

主なオプションには以下があります：

- `--catalog`: 使用するカタログを指定します。設定ファイルに複数のカタログが定義されている場合に、どのカタログを使用するかを明示的に指定できます。
- `--output`: 出力形式を指定します。`text`（人間が読みやすい形式）または`json`（構造化されたデータ形式）が選択できます。特にスクリプトでの自動処理を行う場合は`json`形式が便利です。
- `--uri`: カタログのURIを直接指定します。設定ファイルを使用せずに一時的にカタログに接続する場合に便利です。
- `--credential`: 認証情報を直接指定します。セキュリティ上の理由から、一時的な接続や特定の操作にのみ使用することをお勧めします。
- `--verbose`: 詳細な出力を有効化します。トラブルシューティングや詳細なログが必要な場合に役立ちます。

オプションを明示的に指定しない場合、PyIceberg CLI は `.pyiceberg.yaml` で定義された `default` として定義されたカタログを使用します。これにより、CLI を使用する際に毎回オプションを指定する手間が省けます。

### 主要なコマンド

PyIceberg CLI には、テーブルメタデータを管理するための様々なコマンドが用意されています。

#### 名前空間とテーブルの探索

`list` コマンドは、利用可能な名前空間やテーブルを一覧表示します。引数なしで実行すると名前空間の一覧を、名前空間を指定すると、その名前空間内のテーブル一覧を表示します。

In [ ]:
# 名前空間の一覧を表示
!pyiceberg list

In [ ]:
# 特定の名前空間内のテーブル一覧を表示
!pyiceberg list pyIceberg

`list` コマンドを使えば、どのようなテーブルが利用可能かを簡単に確認できます。また、スクリプトで特定の名前空間内のすべてのテーブルに対して一括処理を行う際にも、このコマンドでテーブル一覧を取得できます。

#### テーブルメタデータの詳細表示

`describe` コマンドは、テーブルの詳細情報を表示します。テーブル形式バージョン、メタデータの場所、UUID、最終更新日時、パーティション仕様、ソート順序、現在のスキーマなど、テーブルの構造と設定に関する包括的な情報が得られます。また、テーブルの最終更新日時を確認してデータの鮮度を把握したり、メタデータファイルの場所を確認して低レベルの調査や問題解決を行ったりする際にも役立ちます。

In [ ]:
!pyiceberg describe pyIceberg.observations

#### テーブルプロパティの管理

`properties` コマンドは、テーブルのプロパティを表示します。

In [ ]:
!pyiceberg properties get table pyIceberg.observations

#### テーブルのロケーション情報の表示

`location` コマンドは、テーブルのベースロケーション（データファイルが格納されている場所）を表示します。これにより、テーブルの物理的な保存場所を確認できます。 

In [ ]:
!pyiceberg location pyIceberg.observations

### テーブルのUUID取得

`uuid` コマンドは、テーブルの一意識別子（UUID）を表示します。この UUID はテーブルを一意に識別するために使用され、テーブル名が変更されても変わりません。

In [ ]:
!pyiceberg uuid pyIceberg.observations

## まとめ
本章では、Python から直接 Iceberg テーブルを操作できる PyIceberg について解説しました。PyIceberg は分散処理環境や Java の実行環境を必要とせず、軽量な環境で Iceberg の高度な機能を活用できるクライアントライブラリです。  
PyIceberg の主な特徴として、ACIDトランザクション、スキーマの進化、タイムトラベルなどの Iceberg の核となる機能を Python 環境から直接利用できる点、そして Pandas、Polars などの主要な Python データ分析ライブラリと統合されている点が挙げられます。これにより、データサイエンティストやアナリスト、アプリケーション開発者が、既存の Python ベースのワークフローを活かしながら Iceberg テーブルを効率的に操作できます。  
PyIceberg は特に、ローカル開発環境でのデータ分析、データサイエンスにおける特徴量エンジニアリング、イベント駆動型データ処理など、様々なユースケースで効果を発揮します。同時に、PyIceberg で管理するテーブルは Spark、Trino などの分散処理エンジンからも操作できるため、データの成長に合わせた柔軟なスケーリングが可能です。  

PyIceberg は軽量なセットアップと低い学習コストで、小規模から中規模のデータを対象とした環境において Iceberg の恩恵を受けるための理想的な入口となり、Python エコシステムとの統合によってより多くのユーザーが Iceberg を活用する可能性を開いています。  